In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [5]:

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [6]:
test=test/255
y_train = train["label"]  # Copy labels
y_train= to_categorical(y_train, num_classes = 10) # One-hot encode the labels
train = train.drop(columns=["label"]) / 255  # Normalize without modifying the original
train_xCNN=train.to_numpy().reshape(-1, 28, 28, 1)

In [7]:
model_CNN = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), strides=2, padding='same', activation='relu'),  # Strided convolution instead of pooling
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),  # Regularization
    
    keras.layers.Flatten(),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.4),  # Regularization
    keras.layers.Dense(10, activation='softmax')  # 10 output classes
])

model_CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary
model_CNN.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 26, 26, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 11, 11, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 6, 6, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 6, 6, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 353,226 (1.35 MB)

 Trainable params: 352,650 (1.35 MB)

 Non-trainable params: 576 (2.25 KB)

In [8]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [9]:
epochs = 45
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.98 ** x)

# Train-test split
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(train_xCNN, y_train, test_size=0.1)  # FIXED

# Train model
history = model_CNN.fit(datagen.flow(X_train2, Y_train2, batch_size=64),  # FIXED
                        epochs=epochs,
                        steps_per_epoch=len(X_train2) // 64,
                        validation_data=(X_val2, Y_val2),
                        callbacks=[annealer],
                        verbose=1)

# Evaluate model
val_loss, val_acc = model_CNN.evaluate(X_val2, Y_val2)
print(f"Validation Accuracy: {val_acc:.4f}")

Epoch 1/45


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


590/590 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - accuracy: 0.7777 - loss: 0.7018 - val_accuracy: 0.9731 - val_loss: 0.1131 - learning_rate: 0.0010
Epoch 2/45
  1/590 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - accuracy: 0.9062 - loss: 0.2103

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9062 - loss: 0.2103 - val_accuracy: 0.9707 - val_loss: 0.1198 - learning_rate: 9.8000e-04
Epoch 3/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 36s 61ms/step - accuracy: 0.9587 - loss: 0.1423 - val_accuracy: 0.9793 - val_loss: 0.0694 - learning_rate: 9.6040e-04
Epoch 4/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9531 - loss: 0.1114 - val_accuracy: 0.9798 - val_loss: 0.0671 - learning_rate: 9.4119e-04
Epoch 5/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 35s 59ms/step - accuracy: 0.9663 - loss: 0.1114 - val_accuracy: 0.9838 - val_loss: 0.0550 - learning_rate: 9.2237e-04
Epoch 6/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9531 - loss: 0.1820 - val_accuracy: 0.9857 - val_loss: 0.0504 - learning_rate: 9.0392e-04
Epoch 7/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 36s 61ms/step - accuracy: 0.9732 - loss: 0.0875 - val_accuracy: 0.9862 - val_loss: 0.0434 - learning_rate: 8.8584e-04
Epoch 8/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.000

In [10]:
model_CNN.save('trained_model.h5')
test_xCNN=test.to_numpy().reshape(-1, 28, 28, 1)

In [17]:

predictions = model_CNN.predict(test_xCNN)
predicted_labels = np.argmax(predictions, axis=1)


875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step


In [18]:
# Create a DataFrame with ImageId and Prediction
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(predicted_labels) + 1),  # Image IDs start from 1
    "Label": predicted_labels
})

# Save to CSV
submission.to_csv("submission.csv", index=False)

print("CSV file 'submission.csv' saved successfully!")


CSV file 'submission.csv' saved successfully!
